In [167]:
import matplotlib.patches as patches
import sys
sys.path.append('..')
sys.path.append('.')
from scipy.signal import correlate
from matplotlib.ticker import FuncFormatter, MultipleLocator
import matplotlib.pyplot as plt
import pickle
import seaborn as sns
from tqdm import tqdm
import random
import argparse
import math
import os
import scipy.io as scio
from scipy.signal import find_peaks
import numpy as np
from scipy import signal
import pandas as pd
import imageio


# Make sequence

In [168]:
T = 1
refreshRate = 60
srate=240

N = int(T*refreshRate)
targetN = 40

colNUM = 8
rowNUM = 5

cubicSize = 140
interval = 50

res_x,res_y = 1600,1200
initWidth, initHeight = 65, 120

freqs = freqs = np.linspace(8.0, 15.8, 40)
phase = [i * np.pi for i in np.tile(np.arange(0, 2+0.5, 0.5),8)]
ts = np.arange(0, T, step=1/refreshRate)
Ns = np.arange(0, T, step=1/srate)
Ns.shape

(240,)

In [169]:
np.random.seed(253)
WN = np.random.random((targetN,N))
SSVEP = np.stack([[np.sin(2*np.pi*f*t+omega) for t in ts]
             for (f, omega) in zip(freqs, phase)])/2+0.5

upWN = np.repeat(WN, srate//refreshRate, axis=-1)
upSSVEP = np.repeat(SSVEP, srate//refreshRate, axis=-1)
upWN.shape

(40, 240)

In [170]:
positions = []
for colINX in range(colNUM):
    for rowINX in range(rowNUM):
        # 左下角
        x = colINX*(cubicSize+interval)+initWidth
        y = (rowNUM-rowINX-1)*(cubicSize+interval)+initHeight

        x = x/res_x
        y = y/res_y

        positions.append((x,y))

In [171]:

for i,(wnV,ssvepV) in tqdm(enumerate(zip(WN.T,SSVEP.T))):

    sns.set_theme(style='ticks')

    fig = plt.figure(figsize=(2*res_x/100, (res_y+500)/100))

    plt.subplots_adjust(wspace=10, hspace=5)
    ax11 = plt.subplot2grid((10, 20), (0, 0), colspan=10, rowspan=7)
    ax12 = plt.subplot2grid((10, 20), (7, 0), colspan=10, rowspan=3)

    ax21 = plt.subplot2grid((10, 20), (0, 10), colspan=10, rowspan=7)
    ax22 = plt.subplot2grid((10, 20), (7, 10), colspan=10, rowspan=3)

    ax11.xaxis.set_major_locator(plt.NullLocator())
    ax11.yaxis.set_major_locator(plt.NullLocator())
    ax21.xaxis.set_major_locator(plt.NullLocator())
    ax21.yaxis.set_major_locator(plt.NullLocator())

    ax11.axis('off')
    ax21.axis('off')

    ax11.set_title('Broadband',fontsize=40)
    ax21.set_title('Narrowband',fontsize=40)


    for pos_i,pos in enumerate(positions):

        target = patches.Rectangle(pos,
                                cubicSize/res_x, cubicSize/res_y,
                                linewidth=1, facecolor=[wnV[pos_i], wnV[pos_i], wnV[pos_i]])

        ax11.add_patch(target)


    for pos_i,pos in enumerate(positions):

        target = patches.Rectangle(pos,
                                cubicSize/res_x, cubicSize/res_y,
                                linewidth=1, facecolor=[ssvepV[pos_i], ssvepV[pos_i], ssvepV[pos_i]])

        ax21.add_patch(target)

    cue = patches.Rectangle(positions[0],
                            (cubicSize)/res_x, (cubicSize)/res_y,
                            linewidth=5, facecolor='none',edgecolor='red')

    ax11.add_patch(cue)
    cue = patches.Rectangle(positions[0],
                            (cubicSize)/res_x, (cubicSize)/res_y,
                            linewidth=5, facecolor='none', edgecolor='red')
    ax21.add_patch(cue)

    ax12.plot(Ns,upWN[0],linewidth=5)
    ax22.plot(Ns, upSSVEP[0], linewidth=5)

    ax12.axvline(x=Ns[i*4], color='red', linewidth=3)
    ax22.axvline(x=Ns[i*4], color='red', linewidth=3)
    ax12.scatter([Ns[i*4]], [upWN[0, i*4]], color='red', s=500)
    ax22.scatter([Ns[i*4]], [upSSVEP[0, i*4]], color='red', s=500)

    for ax in [ax12,ax22]:
        ax.tick_params(labelsize=30, width=0.5, length=2, which='major')
        plt.setp(ax.spines.values(), linewidth=5)
        ax.set_xlabel('Time(s)',fontsize=35)

    sns.despine()
    plt.savefig('frames/%s.png' % i, dpi=100,
                format='png', bbox_inches='tight')
    plt.close()
    # break


60it [01:11,  1.20s/it]


In [172]:
import cv2
import numpy as np
import glob

img_array = []

fps = 60.0
filenames = os.listdir('frames')
filenames.sort(key=lambda x: int(x[:-4]))

for filename in filenames:
    img = cv2.imread('frames'+os.sep+filename)
    height, width, layers = img.shape
    size = (width, height)
    img_array.append(img)

fourcc = cv2.VideoWriter_fourcc(*'XVID')
# fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('project.avi', fourcc, fps,size)

for i in range(len(img_array)):
    out.write(img_array[i])
out.release()
